In [1]:
from dotenv import load_dotenv
import google.generativeai as genai
import pandas as pd
import time
import json
import os
from tqdm.notebook import tqdm

In [2]:
load_dotenv()

True

In [3]:
# 1. 라이브러리 강제 업데이트 (404 에러 해결의 핵심!)
# 실행 후 "Restart Kernel" 버튼이 뜨면 눌러주세요.

# 2. API 키 설정 (본인 키 확인!)
API_KEY = os.getenv('API_KEY')
genai.configure(api_key=API_KEY)

# 3. Role(역할) 정의 - 사용자가 원했던 기능!
# 이 설정이 들어가면 모든 대답을 이 페르소나로 대답합니다.
ROLE_DEFINITION = """
너는 월스트리트에서 20년 경력을 가진 '퀀트 투자 전문가(Quant Analyst)'야.
너의 임무는 뉴스 기사를 읽고 주가에 미칠 영향을 냉철하게 분석하여 점수를 매기는 거야.
감정에 치우치지 말고, 오직 데이터와 팩트에 기반해서 판단해.
"""

# ==========================================
# 2. [핵심] 사용 가능한 모델 자동 찾기 (404 에러 박멸)
valid_model_name = None

print("📡 사용 가능한 모델을 찾는 중...")
try:
    for m in genai.list_models():
        # 'generateContent'를 지원하고 이름에 'flash'가 들어가는 최신 모델 찾기
        if 'generateContent' in m.supported_generation_methods:
            if 'flash' in m.name:
                valid_model_name = m.name
                print(f"✅ 찾았다! 이 모델을 사용합니다: {valid_model_name}")
                break
    
    # 만약 flash가 없으면 pro 모델 찾기
    if not valid_model_name:
        for m in genai.list_models():
            if 'generateContent' in m.supported_generation_methods and 'pro' in m.name:
                valid_model_name = m.name
                print(f"⚠️ Flash 모델을 못 찾아서 Pro 모델로 대체합니다: {valid_model_name}")
                break

    # 모델 설정
    model = genai.GenerativeModel(
        valid_model_name,
        generation_config={"response_mime_type": "application/json"}
    )

except Exception as e:
    print(f"❌ 모델 목록을 가져오는 데 실패했습니다. API 키를 확인해주세요.\n에러: {e}")
    exit()

📡 사용 가능한 모델을 찾는 중...
✅ 찾았다! 이 모델을 사용합니다: models/gemini-2.5-flash


In [ ]:
# 파일 불러오기
file_name = '../newsdata/CJ.csv' 

if not os.path.exists(file_name):
    print("❌ 파일을 찾을 수 없습니다. 같은 폴더에 있는지 확인하세요.")
else:
    # 데이터 읽기
    try:
        df = pd.read_csv(file_name)
    except:
        df = pd.read_csv(file_name, encoding='cp949')

    # 1500개 자르기
    target_df = df[:50].copy()
    print(f"데이터 준비 완료: {len(target_df)}건")

    # 분석 시작
    results_list = []
    print("🚀 퀀트 분석 시작! (Role: 20년차 전문가)")

    for index, row in tqdm(target_df.iterrows(), total=len(target_df)):
        try:
            # 제목, 본문, 종목명 추출
            title = str(row.get('제목', '')) # 컬럼명 확인 필요
            body = str(row.get('본문', ''))[:300]
            ticker = str(row.get('ticker', '해당 종목'))

            # 프롬프트 (Role은 이미 위에서 설정했으므로, 여기선 할 일만 시킴)
            prompt = f"""
            [분석 대상 종목: {ticker}]
            
            [뉴스 데이터]
            제목: {title}
            본문: {body}

            [수행 작업]
            위 뉴스를 보고 호재(1), 중립(0), 악재(-1) 중 하나로 평가해.
            
            [출력 포맷(JSON)]
            {{"score": 정수값, "reason": "이유"}}
            """

            # 실행
            response = model.generate_content(prompt)
            json_res = json.loads(response.text)
            
            results_list.append({
                "original_index": index,
                "score": json_res.get("score"),
                "reason": json_res.get("reason")
            })

            time.sleep(7) # 무료 제한 준수

            # 중간 저장 (50개마다)
            if len(results_list) % 50 == 0:
                pd.DataFrame(results_list).to_csv('temp_results.csv', index=False)

        except Exception as e:
            print(f"⚠️ {index}번 에러: {e}")
            time.sleep(5)

    # 최종 저장
    final_df = pd.merge(target_df, pd.DataFrame(results_list), left_index=True, right_on='original_index', how='left')
    if 'original_index' in final_df.columns:
        final_df = final_df.drop(columns=['original_index'])
        
    final_df.to_csv('final_quant_analysis.csv', index=False, encoding='utf-8-sig')
    print("🎉 저장 완료: final_quant_analysis.csv")
    display(final_df.head())

In [4]:
import pandas as pd
import os

# ==========================================
# 1. 파일 경로 설정
input_path = '../newsdata/CJ.csv' 
# 저장할 파일명 (뒤에 _clean을 붙여 구분)
output_path = '../newsdata/CJ_clean.csv' 

# ==========================================
# 2. 데이터 불러오기
if not os.path.exists(input_path):
    print(f"❌ 파일을 찾을 수 없습니다: {input_path}")
else:
    try:
        df = pd.read_csv(input_path)
    except:
        df = pd.read_csv(input_path, encoding='cp949')

    print(f"📂 원본 데이터 로드 완료: 총 {len(df)}건")

    # ==========================================
    # 3. 중복 제거 (핵심 로직)
    # subset=['제목']: '제목' 컬럼이 같은 것만 찾음
    # keep='first': 중복된 것 중 '가장 위에 있는 것' 하나만 남기고 나머지 삭제
    # (최신순 정렬이 안 되어 있다면 날짜순 정렬 후 실행하는 게 좋습니다)
    
    # 혹시 모를 공백 제거 (예: "삼성전자 " 와 "삼성전자"를 같게 처리)
    if '제목' in df.columns:
        df['제목'] = df['제목'].str.strip() 
        
        # 중복 제거 실행
        df_clean = df.drop_duplicates(subset=['제목'], keep='first')
        
        # 인덱스 재정렬 (깔끔하게 0번부터 다시)
        df_clean = df_clean.reset_index(drop=True)

        # ==========================================
        # 4. 결과 리포트 및 저장
        removed_count = len(df) - len(df_clean)
        
        print("-" * 30)
        print(f"📉 중복 제거 결과:")
        print(f"  - 원본 개수 : {len(df)}개")
        print(f"  - 남은 개수 : {len(df_clean)}개")
        print(f"  - 삭제된 개수: {removed_count}개 (돈 아꼈네요! 💸)")
        print("-" * 30)

        # 저장
        df_clean.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"💾 깨끗한 파일 저장 완료: {output_path}")
        
    else:
        print("🚨 에러: CSV 파일에 '제목'이라는 컬럼이 없습니다. 컬럼명을 확인해주세요.")

📂 원본 데이터 로드 완료: 총 2614건
------------------------------
📉 중복 제거 결과:
  - 원본 개수 : 2614개
  - 남은 개수 : 2438개
  - 삭제된 개수: 176개 (돈 아꼈네요! 💸)
------------------------------
💾 깨끗한 파일 저장 완료: ../newsdata/CJ_clean.csv


In [9]:
df = pd.read_csv('../newsdata/CJ.csv')

In [12]:
df = pd.DataFrame(df)
df.head(100)

,일자,제목,본문,ticker
0,20150101,유통 식음료 업계 신년 화두는 `혁신과 수익창출`,"[이데일리 민재용 기자] 유통 식음료 업체들은 올해 변화와 혁신, 사업 경쟁력 제고...",1040
1,20150102,[투자의맥]박스권 상단 넘어선 종목에 주목,[이데일리 경계영 기자] 대신증권은 2일 올해도 개별 종목 장세가 이어지는 가운데 ...,1040
2,20150108,"CJ프레시웨이, 종합인증우수업체 재공인 획득",[이데일리 염지현 기자] CJ(001040)그룹의 식자재 유통 및 단체급식 전문기업...,1040
3,20150108,올해 창조경제 화두는 '안전'..사회안전과 신시장 창출 노린다,[이데일리 이승현 기자] 정부가 올해 창조경제의 새 화두로 ‘안전’을 내세우며 사회...,1040
4,20150108,"이미경 CJ 부회장, 동생 복귀 도우려 경영서 손 떼나?",[이데일리 민재용 기자] 이미경 CJ(001040)그룹 부회장이 건강상의 이유로 미...,1040
...,...,...,...,...
95,20150417,"코스닥, 7년 3개월 만에 장중 700선 돌파",코스닥지수가 7년 3개월 만에 장중 700선을 돌파한 17일 오전 서울 중구 외환은...,1040
96,20150419,"소외됐던 우선株, 증시 신바람 타고 날아간다",[이데일리 정병묵 기자] 주식 시장이 연일 강세를 보이고 있는 가운데 중대형 우선주...,1040
97,20150420,"CJ, 세계 최대 빈곤퇴치 행사서 '상생활동' 모범사례로 소개",[이데일리 함정선 기자] CJ(001040)그룹은 한국국제협력단(KOICA)과 베트...,1040
98,20150422,"코스닥, 오후 들어 돌연 급락..4% 넘게 '털썩'",[이데일리 김대웅 기자] 코스닥 시장이 새파랗게 물들었다. 외국인과 기관의 공격적인...,1040


In [13]:
import pandas as pd
import os
from difflib import SequenceMatcher
from tqdm import tqdm

# ==========================================
# 1. 설정
input_path = '../newsdata/CJ.csv' 
output_path = '../newsdata/CJ_fuzzy_clean.csv' 
SIMILARITY_THRESHOLD = 0.8  # 80% 이상 비슷하면 삭제

# ==========================================
# 2. 유사도 계산 함수 (0.0 ~ 1.0)
def get_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# ==========================================
# 3. 데이터 로드
if not os.path.exists(input_path):
    print(f"❌ 파일을 찾을 수 없습니다: {input_path}")
else:
    try:
        df = pd.read_csv(input_path)
    except:
        df = pd.read_csv(input_path, encoding='cp949')
    
    # 제목 문자열(String)로 변환 (에러 방지)
    df['제목'] = df['제목'].astype(str)
    
    original_len = len(df)
    print(f"📂 원본 데이터: {original_len}건")

    # ==========================================
    # 4. [핵심 로직] 정렬 후 인접 비교
    # 비슷한 제목끼리 붙여놓기 위해 제목순 정렬
    df = df.sort_values(by='제목').reset_index(drop=True)
    
    indices_to_drop = [] # 삭제할 행 번호를 담을 리스트

    print("🔍 유사한 제목을 찾는 중...")
    
    # 1번째 줄부터 끝까지 돌면서 '바로 윗줄'과 비교
    for i in tqdm(range(1, len(df))):
        prev_title = df.loc[i-1, '제목']
        curr_title = df.loc[i, '제목']
        
        # 유사도 계산
        score = get_similarity(prev_title, curr_title)
        
        # 80% 이상 비슷하면 현재 행(i)을 삭제 목록에 추가
        if score >= SIMILARITY_THRESHOLD:
            indices_to_drop.append(i)

    # ==========================================
    # 5. 삭제 실행 및 저장
    df_clean = df.drop(indices_to_drop)
    
    # (선택) 다시 날짜순이나 원래 순서가 중요하다면 여기서 정렬 필요
    # df_clean = df_clean.sort_values(by='일자') 

    removed_count = len(indices_to_drop)
    
    print("-" * 30)
    print(f"📉 유사도 중복 제거 결과 (기준: {SIMILARITY_THRESHOLD*100}%)")
    print(f"  - 원본 개수 : {original_len}개")
    print(f"  - 삭제된 개수: {removed_count}개")
    print(f"  - 남은 개수 : {len(df_clean)}개")
    print("-" * 30)

    df_clean.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"💾 저장 완료: {output_path}")

📂 원본 데이터: 2614건
🔍 유사한 제목을 찾는 중...


100%|██████████| 2613/2613 [00:00<00:00, 4407.83it/s]


------------------------------
📉 유사도 중복 제거 결과 (기준: 80.0%)
  - 원본 개수 : 2614개
  - 삭제된 개수: 399개
  - 남은 개수 : 2215개
------------------------------
💾 저장 완료: ../newsdata/CJ_fuzzy_clean.csv
